<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'BizFinTech', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-04-29 11:02:29
-------------------
qualified stocks: 93
with latest results: 87
still star stocks: 46


-------------------
Initial Investment:  1.10 C
CY Investment:  1.37 C
Reserve:  2.00 K
Current:  1.18 C
-------------------
Today PnL: 48.86 K (0.42%)
Current PnL: -19.27 L (-14.06%)
CY Booked + Current PnL: -17.73 L (-12.94%)
-------------------
Total profit:  4.22 L
Total loss:  -23.49 L
-------------------
Total Booked + Current PnL: 9.25 L (8.4%)
Total Booked PnL: 28.52 L (25.91%)
Curr Year Booked PnL: 1.54 L (1.3%)
Prev Year Booked PnL: 26.98 L (24.52%)
Est FTT:  1.96 C
Est FTT PnL: 78.06 L (66.15%)
-------------------
Est LTT:  2.57 C
Est LTT PnL: 1.39 C (117.39%)
Deployed:  1.10 C
Current:  1.18 C
CAGR/XIRR %: 4.98%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'XIRR', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SBIN,760.30,863.00,-16.89,M-LC,64.52,217711.0,16232.0,10994.0,0.51,8.06,5.05,13.51,93.0,1.48,1.85,19.27,XY25,NTT,BANKS
32,HDFCBANK,1561.04,2170.00,3.17,X-LC,68.82,100785.0,18050.0,14221.0,-0.97,21.82,14.11,39.01,4.0,1.27,0.85,33.17,X40,BTT,BANKS
70,SBILIFE,1496.49,1924.99,-1.70,H-LC,86.02,145916.0,20211.0,15788.0,0.06,16.08,10.82,28.63,NaN,1.28,1.24,29.99,AR,ATH,INSURANCE
24,DMART,3484.00,5112.00,-16.70,X-LC,67.74,103046.0,19430.0,19641.0,-1.68,23.24,19.06,46.73,33.0,0.99,0.87,26.14,AR,NTT,FMCG
63,RELIANCE,1288.53,1526.00,2.95,X-LC,77.42,220094.0,16506.0,21019.0,1.77,8.11,9.55,18.43,67.0,0.79,1.87,19.89,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,WHIRLPOOL,1112.81,2270.00,-39.46,M-SC,98.92,107986.0,7833.0,96313.0,3.74,7.82,89.19,103.99,34.0,0.08,0.92,30.18,X40,NTT,DURABLES
0,5PAISA,564.75,810.00,-50.63,H-SC,33.33,125449.0,-61483.0,142661.0,3.19,-32.89,113.72,43.43,20.0,-0.43,1.06,20.01,X40N,BTT,FINANCE
11,BANDHANBNK,214.25,400.00,-27.39,H-SC,88.17,196446.0,-52084.0,267559.0,3.10,-20.96,136.20,86.70,-29.0,-0.19,1.67,29.96,XY24,NTT,BANKS
74,SIEMENS,5784.85,7969.85,-52.54,X-LC,2.15,74512.0,-70109.0,124733.0,2.19,-48.48,167.40,37.77,NaN,-0.56,0.63,9.74,AR,ATH,ELECTRICAL
20,COFFEEDAY,59.14,80.00,-128.17,L-SC,87.10,60326.0,-53223.0,93270.0,1.98,-46.87,154.61,35.27,NaN,-0.57,0.51,47.65,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
10,AWL,361.39,485.0,-68.80,H-MC,27.96,137968.0,-48148.0,111809.0,-4.06,-25.87,81.04,34.20,-36.0,-0.43,1.17,13.64,XY24,NTT,FMCG
59,QUESS,711.66,986.0,-64.44,M-SC,1.08,72051.0,-83803.0,143886.0,-2.34,-53.77,199.70,38.55,-33.0,-0.58,0.61,10.33,XY24,NTT,MISC
24,DMART,3484.00,5112.0,-16.70,X-LC,67.74,103046.0,19430.0,19641.0,-1.68,23.24,19.06,46.73,33.0,0.99,0.87,26.14,AR,NTT,FMCG
53,NESTLEIND,2225.87,2755.0,-14.05,X-LC,63.44,216134.0,13580.0,34581.0,-1.28,6.70,16.00,23.77,NaN,0.39,1.83,11.18,XY25,NTT,FMCG
61,RBLBANK,230.61,388.0,-3.18,M-SC,93.55,207996.0,-26304.0,186219.0,-1.15,-11.23,89.53,68.25,-48.0,-0.14,1.76,36.97,XY24,NTT,BANKS


In [12]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,NHPC,82.08,115.00,220.44,M-MC,60.22,149707.0,8694.0,47861.0,0.43,6.17,31.97,40.11,-54.0,0.18,1.27,22.39,XY24,NTT,POWER
55,NMDC,65.62,82.00,35.75,M-MC,6.45,101210.0,811.0,24250.0,0.67,0.81,23.96,24.96,507.0,0.03,0.86,14.07,XY25,NTT,MINING
27,GICRE,335.06,583.00,6.54,H-MC,54.84,188347.0,41926.0,66430.0,0.69,28.63,35.27,74.00,109.0,0.63,1.60,37.74,XY24,BTT,INSURANCE
18,CAMS,3643.00,5250.99,0.88,H-SC,73.12,112269.0,10265.0,34758.0,1.66,10.06,30.96,44.14,261.0,0.30,0.95,31.47,XR,ATH,MISC
4,ADANIPORTS,1103.69,1583.00,-5.49,M-LC,50.54,167058.0,15852.0,49817.0,0.79,10.48,29.82,43.43,NaN,0.32,1.42,15.80,XY24,NTT,MISC


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,0.88,H-SC,73.12,112269.0,10265.0,34758.0,1.66,10.06,30.96,44.14,261.0,0.30,0.95,31.47,XR,ATH,MISC
12,BANKINDIA,112.90,190.00,-25.80,M-MC,66.67,104962.0,4707.0,63754.0,-0.15,4.69,60.74,68.29,-12.0,0.07,0.89,30.82,XR,NTT,BANKS
91,VOLTAS,1272.00,1929.20,-19.13,H-MC,3.23,137632.0,-2288.0,74583.0,0.39,-1.64,54.19,51.67,NaN,-0.03,1.17,3.08,AR,ATH,AC
1,AARTIIND,487.04,919.00,17.20,M-SC,78.49,72653.0,-8683.0,80819.0,0.32,-10.68,111.24,88.69,-8.0,-0.11,0.62,22.51,XR,NTT,CHEMICALS
76,SONACOMS,578.05,1006.00,-36.95,M-MC,44.09,82652.0,-18507.0,93397.0,1.95,-18.29,113.00,74.03,-8.0,-0.20,0.70,16.16,AR,BTT,AUTO


In [14]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HDFCBANK,1561.04,2170.00,3.17,X-LC,68.82,100785.0,18050.0,14221.0,-0.97,21.82,14.11,39.01,4.0,1.27,0.85,33.17,X40,BTT,BANKS
12,BANKINDIA,112.90,190.00,-25.80,M-MC,66.67,104962.0,4707.0,63754.0,-0.15,4.69,60.74,68.29,-12.0,0.07,0.89,30.82,XR,NTT,BANKS
86,UJJIVANSFB,52.77,60.00,-1.02,M-SC,100.00,118125.0,-24354.0,43872.0,-0.18,-17.09,37.14,13.70,-27.0,-0.56,1.00,39.42,X40N,NTT,BANKS
33,HDFCLIFE,562.05,897.00,-3.71,H-LC,42.47,141720.0,29310.0,37683.0,-0.32,26.07,26.59,59.59,16.0,0.78,1.20,33.04,X40,BTT,INSURANCE
70,SBILIFE,1496.49,1924.99,-1.70,H-LC,86.02,145916.0,20211.0,15788.0,0.06,16.08,10.82,28.63,NaN,1.28,1.24,29.99,AR,ATH,INSURANCE


In [15]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,UJJIVANSFB,52.77,60.0,-1.02,M-SC,100.00,118125.0,-24354.0,43872.0,-0.18,-17.09,37.14,13.70,-27.0,-0.56,1.00,39.42,X40N,NTT,BANKS
92,WHIRLPOOL,1112.81,2270.0,-39.46,M-SC,98.92,107986.0,7833.0,96313.0,3.74,7.82,89.19,103.99,34.0,0.08,0.92,30.18,X40,NTT,DURABLES
39,INDUSINDBK,1354.50,1800.0,-35.67,H-MC,97.85,47652.0,-29554.0,54948.0,0.66,-38.28,115.31,32.89,-21.0,-0.54,0.40,31.23,XR,NTT,BANKS
78,SURYODAY,189.78,240.0,-36.78,H-SC,96.77,102202.0,-47724.0,87393.0,1.20,-31.83,85.51,26.46,-41.0,-0.55,0.87,30.93,XR,NTT,BANKS
64,REPCOHOME,515.07,880.0,-13.82,H-SC,95.70,159916.0,-45082.0,190316.0,0.36,-21.99,119.01,70.85,-44.0,-0.24,1.36,29.09,XY24,NTT,FINANCE


In [16]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.68
1,25,41.45
2,50,70.28


In [17]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    38.16
X40     15.37
X40N    12.94
AR      12.33
XY25     9.59
XR       9.27
SR       1.20
X200     1.16
Name: CurrAlloc%, dtype: float64

In [18]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-LC    20.18
H-SC    19.60
M-SC    18.49
M-MC     7.61
H-MC     7.51
M-LC     6.84
H-LC     6.48
X-MC     6.48
X-SC     2.64
L-SC     2.40
Name: CurrAlloc%, dtype: float64

In [19]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
BANKS,12.32,-7.37,59.15
FMCG,10.13,-1.36,32.75
INSURANCE,8.49,4.19,34.44
IT,7.74,-32.88,99.69
FINANCE,7.59,-23.72,79.54
MISC,6.39,-36.68,86.13
PAINTS,5.98,-16.55,51.04
ELECTRICAL,4.29,-16.68,66.13
CHEMICALS,4.13,-11.16,84.42


In [20]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3180717.0,29
X40N,1210443.0,16
XR,1144128.0,14
AR,1043984.0,12
X40,777597.0,13
SR,203460.0,2
XY25,164459.0,6
X200,81200.0,1


In [21]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2205929.0,19
M-SC,2182289.0,22
X-LC,862059.0,14
M-MC,646089.0,7
H-MC,402850.0,7
X-SC,365434.0,4
L-SC,313707.0,4
X-MC,254061.0,5
M-LC,249464.0,5


In [22]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
M-SC       XY24      1013254.0      8
H-SC       XY24       927893.0      6
           X40N       544004.0      6
M-MC       XY24       464688.0      4
M-SC       XR         444645.0      5
X-LC       X40        411366.0      6
H-SC       AR         393977.0      2
M-SC       X40N       376592.0      5
H-SC       XR         290058.0      4
M-LC       XY24       238470.0      4
H-MC       XY24       219496.0      3
X-LC       AR         181970.0      3
M-SC       X40        168770.0      2
L-SC       XR         162912.0      2
X-SC       XR         127811.0      1
           SR         119545.0      1
           X40N       118078.0      2
X-MC       X40N       117546.0      2
H-LC       AR         101205.0      2
X-LC       XY24        97955.0      1
M-SC       AR          95113.0      1
M-MC       AR          93397.0      1
X-LC       XY25        89568.0      3
M-SC       SR          83915.0      1
L-SC       XY24        83837.0      1
X-LC       X200        81200.0      1
H-MC       AR          74583.0      1
L-SC       AR          66958.0      1
M-MC       XR          63754.0      1
X-MC       X40         55958.0      1
H-MC       XR          54948.0      1
H-LC       X40N        54223.0      1
H-MC       X40         53823.0      2
H-SC       X40         49997.0      1
X-MC       XY24        43776.0      1
H-LC       XY25        39647.0      1
           X40         37683.0      1
X-MC       AR          36781.0      1
M-MC       XY25        24250.0      1
M-LC       XY25        10994.0      1

In [23]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
